# Imports

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive
from wmfdata.utils import print_err, pd_display_all

# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/diversity_metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
# METRICS_MONTH_TEXT = '2019-07'
# MEDIAWIKI_HISTORY_SNAPSHOT = '2019-07'

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")


# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_next_month_first_day": str((metrics_month+1).asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics

In [4]:
queries = {
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },

    "global_south_new_editor_retention": {
        "file": "queries/global_south_new_editor_retention.hql",
        "engine": "hive"
    },
    
    "global_south_net_new_content": {
        "file": "queries/global_south_net_new_content.hql",
        "engine": "hive"
    },
    
    "global_south_wikidata": {
        "file": "queries/global_south_net_new_wikidata.hql",
        "engine": "hive"
    },
    
    "global_north_edits_editors": {
        "file": "queries/global_north_edits_editors.hql",
        "engine": "hive"
    },

    "global_north_new_editor_retention": {
        "file": "queries/global_north_new_editor_retention.hql",
        "engine": "hive"
    },
    
    "global_north_net_new_content": {
        "file": "queries/global_north_net_new_content.hql",
        "engine": "hive"
    },
    "global_north_wikidata": {
        "file": "queries/global_north_net_new_wikidata.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running global_north_net_new_content on hive...
Running global_south_edits_editors on hive...
Running global_north_new_editor_retention on hive...
Running global_north_edits_editors on hive...
Running global_north_wikidata on hive...
Running global_south_new_editor_retention on hive...
Running global_south_net_new_content on hive...
Running global_south_wikidata on hive...


# Combining and saving metrics

In [8]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]


In [9]:
# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,global_north_active_editors,global_north_edits,global_north_net_new_content,global_north_new_editor_retention,global_north_nonbot_edits,global_north_wikidata_entities,global_south_active_editors,global_south_edits,global_south_net_new_content,global_south_new_editor_retention,global_south_nonbot_edits,global_south_wikidata_entities
month,,,,,,,,,,,,
"43,739.0","71,113.0","11,650,311.0","1,010,793.0",0.0,"11,650,311.0","332,754.0","20,908.0","2,346,662.0","337,818.0",0.000000,"2,346,370.0","178,675.0"
"43,770.0","78,705.0","24,186,855.0","1,931,146.0",0.0,"24,186,855.0","1,139,864.0","20,692.0","2,203,108.0","393,509.0",0.000000,"2,201,311.0","126,134.0"
"43,800.0","77,246.0","26,570,632.0","2,925,830.0",0.1,"14,433,228.0","1,840,817.0","20,442.0","2,138,211.0","248,453.0",0.044190,"2,203,108.0","49,213.0"
"43,831.0","83,017.0","27,452,425.0","2,549,815.0",0.1,"14,582,826.0","1,536,891.0","21,450.0","2,282,408.0","997,375.0",0.044979,"2,282,408.0","772,105.0"
"43,862.0","71,422.0","23,282,187.0","2,584,204.0",0.1,"10,579,860.0","1,539,784.0","20,245.0","2,191,848.0","489,040.0",0.043097,"1,980,937.0","266,143.0"


In [7]:
metrics.to_csv(FILENAME, sep="\t")